In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
df = pd.read_excel('UAS_RFM.xlsx')
df.head()

,CardID,TrxDate,Amount
0,1000152,2021-01-03,919346
1,1000158,2021-04-23,952179
2,1000156,2021-04-17,86831
3,1000115,2021-02-01,799337
4,1000164,2021-05-06,630839


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 KB


In [4]:
df.isnull().sum()

CardID     0
TrxDate    0
Amount     0
dtype: int64

In [5]:
df.shape

(199, 3)

In [6]:
max_date = max(df['TrxDate'])
max_date

Timestamp('2021-06-29 00:00:00')

In [7]:
df['Diff'] = max_date - df['TrxDate']
df.head()

,CardID,TrxDate,Amount,Diff
0,1000152,2021-01-03,919346,177 days
1,1000158,2021-04-23,952179,67 days
2,1000156,2021-04-17,86831,73 days
3,1000115,2021-02-01,799337,148 days
4,1000164,2021-05-06,630839,54 days


In [8]:
recency = df.groupby('CardID')['Diff'].min().reset_index()
recency.head()

,CardID,Diff
0,1000101,1 days
1,1000102,48 days
2,1000103,105 days
3,1000104,20 days
4,1000105,0 days


In [9]:
recency['Diff'] = recency['Diff'].dt.days
recency.columns = ['CardID','Recency']
recency.head()

,CardID,Recency
0,1000101,1
1,1000102,48
2,1000103,105
3,1000104,20
4,1000105,0


In [10]:
frequency = df.groupby('CardID')['TrxDate'].count().reset_index()
frequency.columns = ['CardID', 'Frequency']
frequency.head()

,CardID,Frequency
0,1000101,1
1,1000102,3
2,1000103,1
3,1000104,2
4,1000105,5


In [11]:
monetary = df.groupby('CardID')['Amount'].sum().reset_index()
monetary.columns = ['CardID', 'Monetary']
monetary.head()

,CardID,Monetary
0,1000101,574210
1,1000102,1608279
2,1000103,792044
3,1000104,733238
4,1000105,2745196


In [12]:
rfm = recency.merge(frequency,how='inner',on=['CardID'])
rfm = rfm.merge(monetary,how='inner',on=['CardID'])
rfm.head()

,CardID,Recency,Frequency,Monetary
0,1000101,1,1,574210
1,1000102,48,3,1608279
2,1000103,105,1,792044
3,1000104,20,2,733238
4,1000105,0,5,2745196


In [13]:
rfm_only = rfm.drop(['CardID'], axis=1)
rfm_only

,Recency,Frequency,Monetary
0,1,1,574210
1,48,3,1608279
2,105,1,792044
3,20,2,733238
4,0,5,2745196
...,...,...,...
66,167,1,684438
67,4,2,1239716
68,78,1,403726
69,89,3,2180562


In [14]:
quantiles = rfm_only.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary
0.25,18.5,2.0,795144.0
0.50,48.0,3.0,1303674.0
0.75,90.0,3.0,2136737.0


In [15]:
quantiles.to_dict()

{'Recency': {0.25: 18.5, 0.5: 48.0, 0.75: 90.0},
 'Frequency': {0.25: 2.0, 0.5: 3.0, 0.75: 3.0},
 'Monetary': {0.25: 795144.0, 0.5: 1303674.0, 0.75: 2136737.0}}

In [16]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [17]:
rfm_segmentation = rfm
rfm_segmentation['R'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

rfm_segmentation.head()

,CardID,Recency,Frequency,Monetary,R,F,M
0,1000101,1,1,574210,4,1,1
1,1000102,48,3,1608279,3,2,3
2,1000103,105,1,792044,1,1,1
3,1000104,20,2,733238,3,1,1
4,1000105,0,5,2745196,4,4,4


In [18]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R.map(str) \
                            + rfm_segmentation.F.map(str) \
                            + rfm_segmentation.M.map(str)
rfm_segmentation.head()

,CardID,Recency,Frequency,Monetary,R,F,M,RFMScore
0,1000101,1,1,574210,4,1,1,411
1,1000102,48,3,1608279,3,2,3,323
2,1000103,105,1,792044,1,1,1,111
3,1000104,20,2,733238,3,1,1,311
4,1000105,0,5,2745196,4,4,4,444


In [19]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('Monetary', ascending=False)

,CardID,Recency,Frequency,Monetary,R,F,M,RFMScore
50,1000152,2,7,5034859,4,4,4,444
55,1000157,4,6,4176369,4,4,4,444
41,1000143,3,7,3275134,4,4,4,444
4,1000105,0,5,2745196,4,4,4,444


In [21]:
print("Customer Terbaik: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Customer Terloyal: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='344']))
print("Customer TerSultan: ",len(rfm_segmentation[rfm_segmentation['M']==4]))
print('Customer Beresiko Tidak Kembali: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Customer Tidak kembali: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Customer tidak kembali dengan Nilai Belanja Kecil: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Customer Terbaik:  4
Customer Terloyal:  5
Customer TerSultan:  18
Customer Beresiko Tidak Kembali:  4
Customer Tidak kembali:  0
Customer tidak kembali dengan Nilai Belanja Kecil:  7
